### Load packages and sample data

In [1]:
import shlex
import re
import pandas as pd
import numpy as np
from typing import List
import utils

In [2]:
fields_to_include = ["OWNER","TITLE", "ABSTRACT", "SERVICETYPE", "SERVICELINK", "KEYWORDS"]
fields_to_output = ["OWNER","TITLE", "SERVICETYPE", "SERVICELINK"]
url_github_repo = "https://github.com/davidoesch/geoservice_harvester_poc/blob/main/data/"
url_github_repo_suffix = "?raw=true"
url_geoservices_CH_csv = "{}geoservices_CH.csv{}".format(url_github_repo,url_github_repo_suffix)
a = 'Strassen auswertung system'

def split_delimiters(word_list_with_delimiters: List[str]) -> List[str]:
    """Take care of left over delimiters, split strings even if in qoutes
        Return a list of words """
    delimiters = [";", ","]

    new_word_list = []

    for word in word_list_with_delimiters:
        if (any(delimiter in word for delimiter in delimiters)):
            splitted_words = re.split(r',|;', word)
            for splitted_word_ in splitted_words:
                new_word_list.append(splitted_word_)
        else:
            new_word_list.append(word)
    return new_word_list

def load_data():
    dataframe = pd.read_csv(url_geoservices_CH_csv, usecols=fields_to_include)
    return dataframe

res = split_delimiters(shlex.split(a))
dataframe_some_cols = load_data()
result = dataframe_some_cols[dataframe_some_cols.apply(lambda dataset: dataset.astype(str).str.contains(res[0], case=False).any(), axis=1)]
database =result.fillna("empty")

### Improvement of the TF-IDF with BM25 to execute a query on the text

In [5]:
# abstracts = result['ABSTRACT'].values.tolist()
bm25 = utils.TFIDF_BM25()
bm25.cleansing_ranking(database, column='ABSTRACT') # 1421 lines in 36 s

In [6]:
# fit the vector to the database
bm25.fit()
# search the best match in the vector
res = bm25.search('Brandmeldeanlage')

In [7]:
for el in res:
    print(result.loc[el])

OWNER                                                      KT_AR
TITLE                                          Feuerwehrplan Gde
ABSTRACT       Der Übersichtsplan der Feuerwehr enthält folge...
KEYWORDS                                                     NaN
SERVICELINK    https://www.geoportal.ch/services/wms/ktar?ser...
SERVICETYPE                                                  WMS
Name: 224, dtype: object


### Keyword extraction with rake_nltk

In [3]:
rake = utils.KeywordsRake()
keywords = rake.extract_keywords(database, column='ABSTRACT', score=False, keyword_length=3)


In [6]:
keywords[0][:8]

['wms service geoportal',
 'kanton appenzell innerrhoden',
 'meteorwasser sowie versickerungsanlagen',
 'karte zeigt',
 'kanalisationsnetz dargestellt',
 'darin ersichtlich',
 'sowie',
 'ölabscheider']

In [7]:
database['ABSTRACT'].values.tolist()[0]

'Diese Karte zeigt die Werkleitungen der Abwasserentsorgung an. Es wird die Strassenentwässerung und Liegenschaftsentwässerung (teilweise mit Hausanschluss) sowie das Kanalisationsnetz dargestellt. Darin ersichtlich sind unter anderem Leitungen mit Schmutzabwasser, Mischabwasser, Regenabwasser, Meteorwasser sowie Versickerungsanlagen, Kontrollschächte, Einlaufschächte und Ölabscheider. WMS Service Geoportal - Kanton Appenzell Innerrhoden'

### Test LSA and LSI with gensim

[^\w\s]


### Implement the classification with spacy and NLP

In [3]:
NLP = utils.NLP_spacy()
keywords = NLP.extract_keywords(database, column='ABSTRACT') # with small models

In [13]:
keywords[2]

[Bundesamtes für Strassen,,
 swisstopo bezogen.,
 Hauptstrassen,
 schweizerischer,
 Hauptstrassen,
 Hauptstrassen,
 Sachverhalten,
 Strassen,
 Geoportal]

In [14]:
database['ABSTRACT'].values.tolist()[2]

'Geodaten des Bundesamtes für Strassen, als WMS der swisstopo bezogen. Der Datensatz der Hauptstrassen umfasst Strassen von allgemein schweizerischer oder internationaler Bedeutung, die nicht als Nationalstrasse definiert sind. Hauptstrassen stellen sowohl die Durchleitung des nationalen und internationalen Verkehrs, als auch die wichtigsten regionalen Verbindungen sicher. Die Achsen der Hauptstrassen können für die Beschreibung und die Referenzierung von Sachverhalten entlang der Strassen verwendet werden. WMS Service Geoportal - Kanton Appenzell Innerrhoden'